In [6]:
import tensorflow.compat.v1 as tf 
import tensorflow.keras as keras
import pandas as pd
import numpy as np
tf.compat.v1.disable_eager_execution()

In [7]:
def normalize(dataset):
    dataset = dataset.apply(lambda x: (x - x.min(axis = 0)) / (x.max(axis = 0) - x.min(axis = 0)))
    dataset = dataset.loc[:,dataset.any()]
    return dataset

In [9]:
dataset = pd.read_csv("2018.csv" , index_col = 0)
dataset = normalize(dataset.iloc[:,:])
# Splitting the dataset into train 70% and test 30%
features = (dataset.shape[1] - 1)
examples = dataset.shape[0]
train_size = int (0.7 * len(dataset))
(x_train , y_train) = (np.array(dataset.iloc[:train_size,:features]), np.array(dataset.iloc[:train_size,features:]))
(x_test , y_test) = (np.array(dataset.iloc[train_size:,:features]), np.array(dataset.iloc[train_size:,features:]))
dataset.head(10)

,DAY,MONTH,DISTANCE,DEP_TIME,ARR_TIME,ORIGIN_ABE,ORIGIN_ABI,ORIGIN_ABQ,ORIGIN_ABR,ORIGIN_ABY,...,DEST_USA,DEST_VEL,DEST_VLD,DEST_VPS,DEST_WRG,DEST_WYS,DEST_XNA,DEST_YAK,DEST_YUM,ARR_DELAY
0,0.066667,0.272727,0.166195,0.709462,0.760734,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.266667,1.000000,0.131058,0.435181,0.474364,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.133333,0.181818,0.041397,0.222176,0.263026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.666667,0.363636,0.059370,0.507295,0.549812,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.166667,0.272727,0.021002,0.550646,0.585244,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.333333,0.545455,0.068457,0.228012,0.300542,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.272727,0.265347,0.762401,0.837849,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.733333,0.090909,0.080775,0.726553,0.807003,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.100000,0.363636,0.251414,0.472697,0.683201,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.533333,0.818182,0.061793,0.273864,0.337641,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# hyper parameters 
learning_rate = 0.0001
epochs = 50
batch_size = 100
batches = int (train_size / batch_size)

In [11]:
x = tf.placeholder(tf.float32 , [None , features])
y = tf.placeholder(tf.float32 , [None , 1])

W = tf.Variable(tf.zeros([features, 1]))
b = tf.Variable(tf.zeros([1]))

pred = tf.nn.sigmoid(tf.matmul(x,W) + b)  

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = pred))

update = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Define accuracy - Percentage of predictions did we get right
correct_prediction = tf.equal(tf.argmax(pred, 1),tf.argmax(y, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

session = tf.Session()
session.run(tf.global_variables_initializer())

for epoch in range(100):     
    for row in range(batches):        
        batch_x = x_train [row * batch_size : (row + 1) * batch_size]
        batch_y = y_train [row * batch_size : (row + 1) * batch_size]
        session.run(update ,feed_dict = {x : batch_x , y : batch_y})
        _, loss_val = session.run([update, loss],feed_dict={x: batch_x, y: batch_y})
        
    print("Iteration: ",epoch, " Accuracy:", session.run(accuracy, feed_dict={x: x_test, y: y_test}),"loss:",loss_val)

Iteration:  0  Accuracy: 1.0 loss: 0.7634338
Iteration:  1  Accuracy: 1.0 loss: 0.7627952
Iteration:  2  Accuracy: 1.0 loss: 0.76216155
Iteration:  3  Accuracy: 1.0 loss: 0.76153296
Iteration:  4  Accuracy: 1.0 loss: 0.76090956
Iteration:  5  Accuracy: 1.0 loss: 0.76029146
Iteration:  6  Accuracy: 1.0 loss: 0.75967866
Iteration:  7  Accuracy: 1.0 loss: 0.7590711
Iteration:  8  Accuracy: 1.0 loss: 0.7584691
Iteration:  9  Accuracy: 1.0 loss: 0.7578724
Iteration:  10  Accuracy: 1.0 loss: 0.75728124
Iteration:  11  Accuracy: 1.0 loss: 0.7566957
Iteration:  12  Accuracy: 1.0 loss: 0.7561157
Iteration:  13  Accuracy: 1.0 loss: 0.7555411
Iteration:  14  Accuracy: 1.0 loss: 0.7549722
Iteration:  15  Accuracy: 1.0 loss: 0.7544091
Iteration:  16  Accuracy: 1.0 loss: 0.7538515
Iteration:  17  Accuracy: 1.0 loss: 0.75329965
Iteration:  18  Accuracy: 1.0 loss: 0.7527533
Iteration:  19  Accuracy: 1.0 loss: 0.7522128
Iteration:  20  Accuracy: 1.0 loss: 0.7516779
Iteration:  21  Accuracy: 1.0 loss: 0

In [13]:
def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

x = dataset.iloc[0:1,:-1]
logistic_fun(np.matmul(np.array(x),session.run(W)) + session.run(b))[0][0]

0.2730646314731332